In [1]:
from gensim.models.doc2vec import *

In [11]:
model = Doc2Vec.load('../store/doc2vec_model_800_food.mod')

In [3]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
vec = cPickle.load(open('../../yelpdata/total_vec_120K_embed.p', 'rb'))

Using TensorFlow backend.


[]


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../yelpdata/total_data_120K.csv')
aspect_columns = ['bit', 'domain']
                 
ds['bit'] = ds['stars'].apply(lambda x : x > 3.0)
train_idxs, val_idxs = train_test_split(ds.index, stratify=ds[['bit', 'domain']], train_size=0.9, random_state=1337)

idxs = val_idxs
H = {}
for aspect in aspect_columns :
    H[aspect] = np.zeros((len(idxs), len(idxs)))
    aspect_vals = ds[aspect].unique()
    for val in aspect_vals :
        a = set(ds[ds[aspect] == val].index) & set(idxs)
        a = map(lambda s : list(idxs).index(s), a)
        for j in a :
            H[aspect][j, a] = 1

    H[aspect][np.arange(len(idxs)), np.arange(len(idxs))] = 0

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
val_X = vec.X[val_idxs]
val_X = map(lambda s : [vec.idx2word[t] for t in s if t > 0], val_X)

In [12]:
embedds = []
for sent in val_X :
    embedds.append(model.infer_vector(sent))

In [13]:
import numpy as np
embedds = np.array(embedds)
embedds.shape

(12709, 800)

In [14]:
from sklearn.preprocessing import normalize
embedds_n = normalize(embedds, 'l2')
scores = np.dot(embedds_n, embedds_n.T)
nb_studies = len(val_idxs)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [10]:
#400
from sklearn.metrics import roc_auc_score

for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

bit 0.550999265872
domain 0.833114392638


In [15]:
from sklearn.metrics import roc_auc_score

for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

bit 0.559887830334
domain 0.832613026223
